In [4]:
import time
import multiprocessing
from pathlib import Path

from torch.utils.data import DataLoader
from utils.dataset import *
from utils.train import *
from utils.utils import *
from models.cnn import *

In [ ]:
model_architecture = CNNModel

In [5]:
dest_dir = '../../../ai_ready/images/'  #
dest_path = Path(dest_dir)
label = '../../../ai_ready/x-ai_data.csv'
model_state_path = './saved_models/pretrained.pth'

BATCH_SIZE = 32
NUM_WORKERS = multiprocessing.cpu_count()
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [6]:
model = model_architecture().to(device)
a
data_transform = transforms.Compose([
    transforms.RandomVerticalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # transforms.RandomRotation(), 
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

train_dataset = SilosImage(dest_path, label, val=False, transform=data_transform)
val_dataset = SilosImage(dest_path, label, val=True, transform=data_transform)




train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=True)

val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=BATCH_SIZE,
                            num_workers=NUM_WORKERS,
                            shuffle=True)


loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [7]:
train(model=model, 
          train_dataloader=train_dataloader,
          test_dataloader=val_dataloader,
          loss_fn=loss_fn,
          optimizer=optimizer,
          epochs=2)

 50%|█████     | 1/2 [00:51<00:51, 51.38s/it]

Epoch: 1 | train_loss: 0.6945 | train_acc: 0.5835 | test_loss: 0.6536 | test_acc: 0.6957


100%|██████████| 2/2 [01:42<00:00, 51.16s/it]

Epoch: 2 | train_loss: 0.6588 | train_acc: 0.6843 | test_loss: 0.6423 | test_acc: 0.6945


{'train_loss': [0.6944659352302551, 0.6587957143783569],
 'train_auc': [0.5834664179436956, 0.6843105793338682],
 'test_loss': [0.6536394953727722, 0.6423413157463074],
 'test_auc': [0.6956773618538324, 0.6945127241179874]}

In [8]:
model_name = 'cnn_' + str(time.time()) + '.pth'
save_model(model, 'saved_models', model_name)

[INFO] Saving model to: save_models\cnn1670422660.415604.pth
